# Lab: Visualizing Learning With Tensorboard

### Overview
Introducing a suit of visualization tools

### Runtime
30 mins


# MNIST Dataset Introduction

Most examples are using MNIST dataset of handwritten digits. The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1. For simplicity, each image has been flatten and converted to a 1-D numpy array of 784 features (28*28).

## Overview

![MNIST Digits](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

## Usage
In our examples, we are using TensorFlow [input_data.py](https://github.com/tensorflow/tensorflow/blob/r0.7/tensorflow/examples/tutorials/mnist/input_data.py) script to load that dataset.
It is quite useful for managing our data, and handle:

- Dataset downloading

- Loading the entire dataset into numpy array: 




#### In this section we are going to download the dataset from the website

In [3]:
from __future__ import print_function

import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

C:\Users\hp\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


#### Then we need to set and define all parameteres needed for the lab

In [4]:
learn_rate = 0.01                                                              
train_epochs = 25                                                           
batch_size = 100
display_epoch = 1
logs_path = '/tmp/tensorflow_logs/example/'

# mnist data image : 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='Input')

# We have 0-9 digits 
y = tf.placeholder(tf.float32, [None, 10], name='Label')

# Setting model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weight')
b = tf.Variable(tf.zeros([10]), name='Bias')

#### Now it's time to construct the model. Encapsulation will be applied for TensorBoard graph 

In [6]:
with tf.name_scope('Model'):
    # Model with softmax algorithm
    pred = tf.nn.softmax(tf.matmul(x, W) + b)
    
with tf.name_scope('Loss'):
    # Minimizing error using cross entropy
    cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
    
with tf.name_scope('SGD'):
    # Using gradient descent
    optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(cost)
    
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Creating a summary to monitor cost tensor
tf.summary.scalar("loss", cost)

# Creating a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)

# Merging all summaries into a single op
merged_summary_op = tf.summary.merge_all()

### Training time

In [9]:
with tf.Session() as sess:
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(train_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            
            # Writing logs at each iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            
            # Compute average loss
            avg_cost += c / total_batch
            
        # Displaying logs per epoch step
        if (epoch+1) % display_epoch == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

Epoch: 0001 cost= 1.183676555
Epoch: 0002 cost= 0.665286418
Epoch: 0003 cost= 0.552750317
Epoch: 0004 cost= 0.498687048
Epoch: 0005 cost= 0.465503496
Epoch: 0006 cost= 0.442538138
Epoch: 0007 cost= 0.425545033
Epoch: 0008 cost= 0.412181223
Epoch: 0009 cost= 0.401358804
Epoch: 0010 cost= 0.392405644
Epoch: 0011 cost= 0.384789361
Epoch: 0012 cost= 0.378139623
Epoch: 0013 cost= 0.372448440
Epoch: 0014 cost= 0.367326876
Epoch: 0015 cost= 0.362731803
Epoch: 0016 cost= 0.358596875
Epoch: 0017 cost= 0.354868980
Epoch: 0018 cost= 0.351477642
Epoch: 0019 cost= 0.348303930
Epoch: 0020 cost= 0.345434611
Epoch: 0021 cost= 0.342764543
Epoch: 0022 cost= 0.340234761
Epoch: 0023 cost= 0.337917723
Epoch: 0024 cost= 0.335750424
Epoch: 0025 cost= 0.333718045
Optimization Finished!
Accuracy: 0.914
Run the command line:
--> tensorboard --logdir=/tmp/tensorflow_logs 
Then open http://0.0.0.0:6006/ into your web browser


### Testing the model and calculating accuracy

In [ ]:
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

### Now we run the TensorBaord

Now run the following command on a bash

```bash
tensorboard --logdir=/tmp/tensorflow_logs
```
Then open the browser(Chrome) and insert this address

```bash
localhost:6006
```